# 使用``HybridBlock``更快和更好移植

* 命令式编程的优点：我们会对程序的执行流程很清楚，程序会很灵活并且我们可以尽情的通过print来调试，我们可以通过增加循环和条件判断来增加程序的逻辑性。

* 符号式编程的优点：但是命令式变成的一大缺点是速度会相对较慢，这就是符号式变成所要带来的改变，一般命令式编程是执行一句就分配一块内存，这样带来的问题可能是说也许前面的某个变量我们在之后根本就用不到，因为他可能只是作为中间变量来过渡结果，因此通过命令式编程我们的程序是认识不到这点的，符号式编程是说我们会将整个程序的逻辑定义好最后再一同编译，这样的一个好处是说可以看清程序的全貌之后我们便可以只分配必须的内存，因此通过符号式编程会带来性能的提升。

In [1]:
import mxnet as mx

from mxnet import nd

In [2]:
def add_two(A, B):
    return A + B

def add(A, B, C, D):
    E = add_two(A, B)
    F = add_two(C, D)
    return E + F

In [3]:
add(3, 4, 5, 6)

18

In [4]:
def fancy_add_two():
    return '''
def add_two(A, B):
    return A + B    
    '''

def fancy_add():
    return '''
def add(A, B, C, D):
    E = add_two(A, B)
    F = add_two(C, D)
    return E + F   
    '''

def evoke_add():
    return fancy_add_two() + fancy_add() + \
'''
print(add(1, 2, 3, 4))
'''
    
evoke = evoke_add()
y = compile(evoke, '', 'exec')
exec(y)

10


## 使用``HybridBlock``,``HybridSequential``,``.hybridize()``来加速

In [5]:
from mxnet import gluon

def get_net():
    net = gluon.nn.HybridSequential()
    with net.name_scope():
        net.add(gluon.nn.Dense(256))
        net.add(gluon.nn.Dense(128))
        net.add(gluon.nn.Dense(10))
    net.initialize()
    return net

In [6]:
net = get_net()
X = nd.random.uniform(shape=(1, 784))
net(X)


[[ 0.22535166  0.04538546 -0.05891204 -0.10385949 -0.17111427 -0.06184213
  -0.03462029  0.07846655 -0.49402252  0.07973635]]
<NDArray 1x10 @cpu(0)>

In [7]:
net.hybridize()
net(X)


[[ 0.22535166  0.04538546 -0.05891204 -0.10385949 -0.17111427 -0.06184213
  -0.03462029  0.07846655 -0.49402252  0.07973635]]
<NDArray 1x10 @cpu(0)>

我们来看看hybridize比普通的initialize快几倍

In [11]:
from time import time 

def bench(net, X):
    start = time()
    for i in range(1000):
        y = net(X)
    nd.waitall()
    return time() - start

net = get_net()
print("Before Hybridize : ", bench(net, X))
net.hybridize()
print("After Hybridize : ", bench(net, X))

Before Hybridize :  0.5158853530883789
After Hybridize :  0.3645961284637451


可以看到hybridize还是快很多的

下面我们继承``HybridizeBlock``来自定义网络

## ``Symbol``

In [15]:
from mxnet import sym

X = sym.var('data')
print(X)

y = net(X)
print(y)

y_json = y.tojson()
print(y_json)

<Symbol data>
<Symbol hybridsequential3_dense2_fwd>
{
  "nodes": [
    {
      "op": "null", 
      "name": "data", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "hybridsequential3_dense0_weight", 
      "attrs": {
        "__dtype__": "0", 
        "__lr_mult__": "1.0", 
        "__shape__": "(256, 0)", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "hybridsequential3_dense0_bias", 
      "attrs": {
        "__dtype__": "0", 
        "__init__": "zeros", 
        "__lr_mult__": "1.0", 
        "__shape__": "(256,)", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "FullyConnected", 
      "name": "hybridsequential3_dense0_fwd", 
      "attrs": {
        "flatten": "True", 
        "no_bias": "False", 
        "num_hidden": "256"
      }, 
      "inputs": [[0, 0, 0], [1, 0, 0], [2, 0, 0]]
    }, 
    {
      "op": "null", 
      "name": "hybridsequential3_dense1_weigh

In [16]:
y.save('model.json')

In [17]:
net.save_params('model.params')

In [22]:
class FancyHybridBlock(gluon.HybridBlock):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        with self.name_scope():
            self.fc1 = gluon.nn.Dense(256)
            self.fc2 = gluon.nn.Dense(128)
            self.fc3 = gluon.nn.Dense(2) 
        
    def hybrid_forward(self, F, X):
        # F is a function space that depends on the type of x
        # If x's type is NDArray, then F will be mxnet.nd
        # If x's type is Symbol, then F will be mxnet.sym
        print("type(X) : {}, F : {}".format(type(X).__name__, F.__name__))
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        return self.fc3(X)

In [28]:
blk = FancyHybridBlock()
blk.collect_params().initialize()

In [30]:
x = nd.random.normal(shape=(1,512))
print("===first===")
y = blk(x)
print("===second===")
y = blk(x)

===first===
type(X) : NDArray, F : mxnet.ndarray
===second===
type(X) : NDArray, F : mxnet.ndarray


In [31]:
blk.hybridize()
print("===first===")
y = blk(x)
print("===second===")
y = blk(x)

===first===
type(X) : Symbol, F : mxnet.symbol
===second===


可以看到什么使用hubridize后第二次什么都没有输出。这是因为第一次net(x)的时候，会先将输入替换成Symbol来构建符号式的程序，之后运行的时候系统将不再访问Python的代码，而是直接在C++后端执行这个符号式程序。这是为什么hybridze后会变快的一个原因。

但这种速度的提升的一个问题就是他不够灵活，我们无法多次执行forward来跑我们的结果，